In [2]:
pip install torch_sparse

  Created wheel for torch-sparse: filename=torch_sparse-0.6.8-cp36-cp36m-linux_x86_64.whl size=22582506 sha256=79d0ef679e44fc37e00c952481c091d122defe5d1468f7790ee9ba2fdc16fa80
  Stored in directory: /root/.cache/pip/wheels/31/d3/94/e2560941994bebe665b1dccc69faf33d33aeee2bca353c9bcc
Successfully built torch-sparse


In [5]:
pip install torch_scatter

  Created wheel for torch-scatter: filename=torch_scatter-2.0.5-cp36-cp36m-linux_x86_64.whl size=9481338 sha256=e0f165d56d428c26833d3f3cf77f06d4ba0045a8cfd565b1b04e533c5880fd95
  Stored in directory: /root/.cache/pip/wheels/42/59/d3/9866e4fd8e1fe9260740acfe22322c428bc0dc064d3ebc456c
Successfully built torch-scatter


In [1]:
pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 torch-cluster==latest+cu101 torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     |████████████████████████████████| 11.9MB 2.1MB/s 
  Using cached https://pytorch-geometric.com/whl/torch-1.7.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://pytorch-geometric.com/whl/torch-1.7.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://pytorch-geometric.com/whl/torch-1.7.0/torch_spline_conv-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5
  Found existing installation: torch-sparse 0.6.8
    Uninstalling torch-sparse-0.6.8:
      Successfully uninstalled torch-sparse-0.6.8
  Found existing installation: torch-cluster 1.5.8
    Uninstalling torch-cluster-1.5.8:
      Successfully uninstalled torch-cluster-1.5.8
  Found existing installation: torch-spline-conv 1.2.0
    Uninstalling torch-spline-conv-1.2.0:
    

In [13]:
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset

In [15]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

In [3]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [5]:
len(dataset)

600

In [6]:
dataset.num_classes

6

In [7]:
dataset.num_node_features

3

In [8]:
data.is_undirected()

True

In [11]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]

In [14]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [18]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [20]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [22]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8060
